##### Copyright 2021 The TensorFlow Cloud Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Setting Up and Connecting To Your Google Cloud Account

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/cloud/tutorials/google_cloud_project_setup_instructions.ipynb"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/cloud/blob/master/g3doc/tutorials/google_cloud_project_setup_instructions.ipynb""><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/cloud/blob/master/g3doc/tutorials/google_cloud_project_setup_instructions.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/cloud/tutorials/google_cloud_project_setup_instructions.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
        <a href="https://kaggle.com/kernels/welcome?src=https://github.com/tensorflow/cloud/blob/master/g3doc/tutorials/google_cloud_project_setup_instructions.ipynb" target="blank">
            <img width="90" src="https://www.kaggle.com/static/images/site-logo.png" alt="Kaggle logo">Run in Kaggle
        </a>
     </td>
</table>


This guide is to help first time users set up a Google Cloud Platform account specifically with the intention to use [tensorflow_cloud](https://tensorflow.org/cloud) to easily run training at scale on Google Cloud AI Platform. [TensorFlow Cloud](https://tensorflow.org/cloud) provides APIs that allow users to easily go from debugging, training, tuning Keras and TensorFlow code in a local or kaggle environment to distributed training/tuning on Cloud.

## 1. Sign up for Google Cloud Platform

To start go to https://cloud.google.com/ and click on “Get Started For Free". This is a two step sign up process where you will need to provide your name, address and a credit card. The starter account is free and it comes with $300 credit that you can use. For this step you will need to provide a Google Account ( i.e. your Gmail account) to sign in.

After completing the sign up process you will be redirected to [Google Cloud Platform welcome page](https://console.cloud.google.com/home/dashboard). click on the "Home" tab and make a note of your Project ID and Project number. (see [Identifying projects](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects))

In [1]:
GCP_PROJECT_ID = 'YOUR_PROJECT_ID'
PROJECT_NUMBER = 'YOUR_PROJECT_NUMBER'

## 2.Enable Google Cloud SDK and Authenticate your notebook
Now that you have set up your project we can continue with the rest of the configuration steps directly from this notebook. There Three types of notebook, this step (step #2) is slightly different for each notebook, the rest of the steps (step #3 to #6) are the same for all notebooks.

* 2.1. Auth for Kaggle notebooks
* 2.2. Auth for Colab notebook
* 2.3. Auth for Cloud AI Notebooks - Not supported.

### 2.1 Setup Auth for a Kaggle Notebook

If you are using a Kaggle notebook you will need to repeat this step for any new notebook that you use with this project. In your notebook click on Add-ons -> Google Cloud SDK and follow the instructions on the prompt. Then run the following command to add your auth credentials to this notebook.

In [ ]:
import sys
if "kaggle_secrets" in sys.modules:
    from kaggle_secrets import UserSecretsClient
    UserSecretsClient().set_gcloud_credentials(project=GCP_PROJECT_ID)

### 2.2 Setup Auth for a Colab Notebook
If you are using a Colab notebook you will need to repeat this step for any new notebook that you use with this project. Run the following command to add your auth credentials to this notebook.

In [ ]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## 3. Link your billing account to your project
Next step is to [set up  the billing account](https://cloud.google.com/billing/docs/how-to/modify-project) for this project. Google Cloud Creates a project for you by default which is called “My First Project”. We will use this default project. Use your Project ID (from step 1) to run the following commands. This will show you your Billing Account_ID, make a note of this for the next step.

In [ ]:
!gcloud beta billing accounts list

Use your Billing Account_ID from above and run the following to link your billing account with your project. 

Note if you use an existing project you may not see an Account_ID, this means you do not have the proper permissions to run the following commands, contact your admin or create a new project.

In [ ]:
BILLING_ACCOUNT_ID = 'YOUR_BILLING_ACCOUNT_ID'

!gcloud beta billing projects link $GCP_PROJECT_ID --billing-account $BILLING_ACCOUNT_ID

## 4. Enable Required APIs for tensorflow-cloud in your project
For tensorflow_cloud we use two specific APIs: [AI Platform Training Jobs API](https://console.cloud.google.com/marketplace/product/google/ml.googleapis.com) and [Cloud builder API]( https://console.developers.google.com/apis/api/cloudbuild.googleapis.com/overview). Note that this is a one time setup for this project, you do not need to rerun this command for every notebook.


In [ ]:
!gcloud services --project $GCP_PROJECT_ID enable ml.googleapis.com cloudbuild.googleapis.com

## 5. Create a Google Cloud Storage bucket

We will use this storage bucket for temporary assets as well as to save the model checkpoints. Make a note of the name of the bucket for future reference. Note bucket names are unique globally.

In [ ]:
BUCKET_NAME = 'YOUR_BUCKET_NAME'

GCS_BUCKET = f'gs://{BUCKET_NAME}'
!gsutil mb -p $GCP_PROJECT_ID $GCS_BUCKET

## Create a service account for HP Tuning jobs
This step is required to use HP Tuning on Google Cloud using CloudTuner.
To [create a service account](https://cloud.google.com/iam/docs/creating-managing-service-accounts#iam-service-accounts-create-gcloud) and give it project editor access run the following command and make a note of your service account name.

In [ ]:
SERVICE_ACCOUNT_NAME ='YOUR_SERVICE_ACCOUNT_NAME'

SERVICE_ACCOUNT_EMAIL = f'{SERVICE_ACCOUNT_NAME}@{GCP_PROJECT_ID}.iam.gserviceaccount.com'
!gcloud iam --project $GCP_PROJECT_ID service-accounts create $SERVICE_ACCOUNT_NAME
!gcloud projects add-iam-policy-binding $GCP_PROJECT_ID \
    --member serviceAccount:$SERVICE_ACCOUNT_EMAIL \
    --role=roles/editor

The [`default AI Platform service account`](https://cloud.google.com/ai-platform/training/docs/custom-service-account#default) is identified by an email address with the format `service-PROJECT_NUMBER@cloud-ml.google.com.iam.gserviceaccount.com`. Using your Project number from step one, we construct the service account email and grant the [`default AI Platform service account`](https://cloud.google.com/ai-platform/training/docs/custom-service-account#default) admin role (roles/iam.serviceAccountAdmin) on your new service account.

In [ ]:
DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT = f'service-{PROJECT_NUMBER}@cloud-ml.google.com.iam.gserviceaccount.com'

!gcloud iam --project $GCP_PROJECT_ID service-accounts add-iam-policy-binding \
--role=roles/iam.serviceAccountAdmin \
--member=serviceAccount:$DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT \
$SERVICE_ACCOUNT_EMAIL

## Congratulations !
You are now ready to run tensorflow-cloud. Note that these steps only need to be run one time. Once you have your project setup you can reuse the same project and bucket configuration for future runs. For any new notebooks you will need to repeat the step two to add your Google Cloud auth credentials.

Make a note of the following values as they are needed to run tensorflow-cloud.

In [ ]:
print(f"Your GCP_PROJECT_ID is:       {GCP_PROJECT_ID}")
print(f"Your SERVICE_ACCOUNT_NAME is: {SERVICE_ACCOUNT_NAME}")
print(f"Your BUCKET_NAME is:          {BUCKET_NAME}")

Your GCP_PROJECT_ID is:       YOUR_PROJECT_ID
Your SERVICE_ACCOUNT_NAME is: YOUR_SERVICE_ACCOUNT_NAME
Your BUCKET_NAME is:          YOUR_BUCKET_NAME
